# STAGE 1: Get data from GitHub to DB SQLite

In [ ]:
import git
SOURCE_URL = 'https://github.com/CSSEGISandData/COVID-19'
DATA_PATH = '/data'
git.Repo.clone_from(SOURCE_URL, DATA_PATH)

## Import

In [68]:
import sqlite3 as sl
import csv
import os
import re
from re import search



## Functions and variables

In [70]:


def GetValuesPart(text):
    return re.sub('\w+[^,]', '?', text)

def ReplaceFields(text, fieldReplaceStorage):
    for key in fieldReplaceStorage:
        text = text.replace(key, fieldReplaceStorage[key])
    return text  

def CreateTableInDB(nameDB, query, queryUS):
    with con:
        cur.execute(query)
        cur.execute(queryUS)

def GetFileNameList(path):
    result = []
    for root, dirs, files in os.walk(path): 
        for file in files: 
            result.append(os.path.join(root,file).replace("\\", "/"))
    return result        
    
def ReplaceInList(row, replaceable, on):
    result = []
    for element in row:
        element = element.replace(replaceable, on)
        result.append(element)
    return result    

def GetFileDateFromName(name):
    return re.search('\d+-\d+-\d+', name).group() 

def AddFileDateFieldToText(headRow):
    if search("\w+", headRow):
        return headRow + ",File_Date"
    else:
      return "File_Date"

def ImportDataFromFileReaderToDB(fileReader, fileDate, insertQuery):
    for row in fileReader:
        row = ReplaceInList(row, "/", "-")
        row.append(fileDate)
        cur.execute(insertQuery, row)

def GetHeadRowFromFileReader(fileReader):
    headRow = ",".join(next(fileReader))
    headRow = ReplaceFields(headRow, fieldReplaceStorage) 
    return headRow

def ImportDataFromFilesToDB(fileNameList, nameTable):
    for name in fileNameList:
        if (".csv" in name):
            with open(name, encoding='utf-8') as r_file:
                fileReader = csv.reader(r_file, delimiter = ",")
                headRow = AddFileDateFieldToText(GetHeadRowFromFileReader(fileReader))
                valuesPart = GetValuesPart(headRow)
                insertQuery = f"INSERT INTO {nameTable} ({headRow}) values({valuesPart}) " 
                fileDate = GetFileDateFromName(name) 
                ImportDataFromFileReaderToDB(fileReader, fileDate, insertQuery)  
                                    

def ImportDataFromFoldersToDB(nameTable, folderPathes):
    try:
        fileNameList = GetFileNameList(folderPathes[0])
        ImportDataFromFilesToDB(fileNameList, nameTable)
        fileNameList = GetFileNameList(folderPathes[1])
        ImportDataFromFilesToDB(fileNameList, nameTableUS)               
        con.commit()
    
    finally:
        if cur is not None:
            cur.close()

        if con is not None:
            con.close()

fieldReplaceStorage = {
    "Incidence_Rate": "Incident_Rate",
    "Lat,": "Latitude,",
    "Long_": "Longitude",
    "/": "_",
    " ": "_",
    "-": "_"
}

nameTable = "statistics"
nameTableUS = "statisticsUS"
nameDB = "corona_statistics.db"
path1 = "D:\GitHub\DA\csse_covid_19_data\csse_covid_19_daily_reports"
path2 = "D:\GitHub\DA\csse_covid_19_data\csse_covid_19_daily_reports_us"
pathList = [path1, path2]

query = """
        CREATE TABLE statistics (
        FIPS                 INT,
        Admin2               TEXT,
        Province_State       TEXT,
        Country_Region       TEXT,
        Last_Update          DATETIME,
        Latitude             REAL,
        Longitude            REAL,
        Confirmed            INT, 
        Deaths               INT, 
        Recovered            INT, 
        Active               INT, 
        Combined_Key         TEXT,
        Incident_Rate        REAL,
        Case_Fatality_Ratio  REAL,
        File_Date            DATE
    );
    """


queryUS = """
        CREATE TABLE statisticsUS (
        FIPS                 INT,
        Province_State       TEXT,
        Country_Region       TEXT,
        Last_Update          DATETIME,
        Latitude             REAL, 
        Longitude            REAL,
        Confirmed            INT, 
        Deaths               INT, 
        Recovered            INT, 
        Active               INT, 
        Incident_Rate        REAL,
        Case_Fatality_Ratio  REAL,
        Total_Test_Results   REAL,
        People_Hospitalized  INT,
        UID                  INT,
        ISO3                 TEXT,
        Testing_Rate         REAL,
        Hospitalization_Rate REAL,
        People_Tested        INT,
        Mortality_Rate       REAL,
        Cases_28_Days        REAL,
        Deaths_28_Days       REAL,
        File_Date            DATE
    );
    """

## Connection

In [71]:
con = sl.connect(nameDB)  
cur = con.cursor()  

## Create table

In [72]:
CreateTableInDB(nameDB, query, queryUS)

## Import data to SQLite

In [73]:
ImportDataFromFoldersToDB(nameTable, pathList)

# STAGE 2: Data processing

## Import

In [74]:
import pandas as pd
import sqlite3

## Connection

In [75]:
con = sqlite3.connect("corona_statistics.db")
cur = con.cursor()


## Read all data from reports and reports_us in SQLite

In [76]:
query = "SELECT * FROM statistics"
queryUS = "SELECT * FROM statisticsUS"
df = pd.read_sql(query, con)
dfUS = pd.read_sql(queryUS, con)

## Getting an idea of the data in columns

#### Reports

In [77]:
df.head(100)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Latitude,Longitude,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,File_Date
0,,,,Afghanistan,2021-01-02 05:22:33,33.93911,67.709953,52513,2201,41727,0,Afghanistan,0.0,4.252222,01-01-2021
1,,,,Albania,2021-01-02 05:22:33,41.1533,20.1683,58316,1181,33634,23501,Albania,2026.409062,2.025173,01-01-2021
2,,,,Algeria,2021-01-02 05:22:33,28.0339,1.6596,99897,2762,67395,29740,Algeria,227.809861,2.764848,01-01-2021
3,,,,Andorra,2021-01-02 05:22:33,42.5063,1.5218,8117,84,7463,570,Andorra,10505.403482,1.034865,01-01-2021
4,,,,Angola,2021-01-02 05:22:33,-11.2027,17.8739,17568,405,11146,6017,Angola,53.452981,2.305328,01-01-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,,,Araucania,Chile,2021-01-02 05:22:33,-38.9489,-72.3311,23161,311,21736,1114,"Araucania, Chile",2419.600846,1.342774,01-01-2021
96,,,Arica y Parinacota,Chile,2021-01-02 05:22:33,-18.594,-69.4785,10859,225,10408,226,"Arica y Parinacota, Chile",4803.421979,2.072014,01-01-2021
97,,,Atacama,Chile,2021-01-02 05:22:33,-27.5661,-70.0503,8520,113,8328,79,"Atacama, Chile",2948.668254,1.326291,01-01-2021
98,,,Aysen,Chile,2021-01-02 05:22:33,-45.9864,-73.7669,1588,15,1436,137,"Aysen, Chile",1539.386184,0.944584,01-01-2021


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199731 entries, 0 to 2199730
Data columns (total 15 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   FIPS                 object
 1   Admin2               object
 2   Province_State       object
 3   Country_Region       object
 4   Last_Update          object
 5   Latitude             object
 6   Longitude            object
 7   Confirmed            object
 8   Deaths               object
 9   Recovered            object
 10  Active               object
 11  Combined_Key         object
 12  Incident_Rate        object
 13  Case_Fatality_Ratio  object
 14  File_Date            object
dtypes: object(15)
memory usage: 251.7+ MB


#### Reports_US

In [79]:
dfUS.head(100)

,FIPS,Province_State,Country_Region,Last_Update,Latitude,Longitude,Confirmed,Deaths,Recovered,Active,...,People_Hospitalized,UID,ISO3,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate,Cases_28_Days,Deaths_28_Days,File_Date
0,1,Alabama,US,2021-01-02 05:30:44,32.3182,-86.9023,365747,4872,202137,158738,...,,84000001,USA,38448.804196,,None,None,NaN,NaN,01-01-2021
1,2,Alaska,US,2021-01-02 05:30:44,61.3707,-152.4044,47019,206,7165,39615,...,,84000002,USA,174391.185778,,None,None,NaN,NaN,01-01-2021
2,60,American Samoa,US,2021-01-02 05:30:44,-14.271,-170.132,0,0,,0,...,,16,ASM,3846.084722,,None,None,NaN,NaN,01-01-2021
3,4,Arizona,US,2021-01-02 05:30:44,33.7298,-111.4312,530267,9015,76934,444318,...,,84000004,USA,39551.860582,,None,None,NaN,NaN,01-01-2021
4,5,Arkansas,US,2021-01-02 05:30:44,34.9697,-92.3731,229442,3711,199247,26484,...,,84000005,USA,67979.497674,,None,None,NaN,NaN,01-01-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,37,North Carolina,US,2021-01-03 05:30:33,35.6301,-79.8064,558437,6892,403488,148057,...,,84000037,USA,67499.306832,,None,None,NaN,NaN,01-02-2021
96,38,North Dakota,US,2021-01-03 05:30:33,47.5289,-99.784,92891,1317,89582,1999,...,,84000038,USA,169287.669507,,None,None,NaN,NaN,01-02-2021
97,69,Northern Mariana Islands,US,2021-01-03 05:30:33,15.0979,145.6739,122,2,29,91,...,,580,MNP,31827.578703,,None,None,NaN,NaN,01-02-2021
98,39,Ohio,US,2021-01-03 05:30:33,40.3888,-82.7649,714673,13919,573641,132015,...,,84000039,USA,66891.445877,,None,None,NaN,NaN,01-02-2021


In [38]:
dfUS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31570 entries, 0 to 31569
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   FIPS                  31570 non-null  object 
 1   Province_State        31570 non-null  object 
 2   Country_Region        31570 non-null  object 
 3   Last_Update           31570 non-null  object 
 4   Latitude              31570 non-null  object 
 5   Longitude             31570 non-null  object 
 6   Confirmed             31570 non-null  int64  
 7   Deaths                31570 non-null  int64  
 8   Recovered             31570 non-null  object 
 9   Active                31570 non-null  object 
 10  Incident_Rate         31570 non-null  object 
 11  Case_Fatality_Ratio   19314 non-null  object 
 12  Total_Test_Results    19314 non-null  object 
 13  People_Hospitalized   31570 non-null  object 
 14  UID                   31570 non-null  int64  
 15  ISO3               

## Set the correct data type to the columns:

In [80]:
def SetStringType(df, *columnNames):
    for name in columnNames:
        df[name] = df[name].astype("string")

def SetNumericType(df, *columnNames):
    for name in columnNames:
        dfUS[name] = pd.to_numeric(name)        

#### Reports

In [81]:

df["Last_Update"]         = pd.to_datetime(df["Last_Update"])
df['File_Date']           = pd.to_datetime(df['File_Date'])
df["FIPS"]                = pd.to_numeric(df["FIPS"])
df["Confirmed"]           = pd.to_numeric(df["Confirmed"])
df["Deaths"]              = pd.to_numeric(df["Deaths"])
df["Recovered"]           = pd.to_numeric(df["Recovered"])
df["Active"]              = pd.to_numeric(df["Active"])
df["Latitude"]            = pd.to_numeric(df["Latitude"])
df["Longitude"]           = pd.to_numeric(df["Longitude"])
df["Incident_Rate"]       = pd.to_numeric(df["Incident_Rate"])

SetStringType(df, "Admin2", "Province_State", "Country_Region")

Reports-US

In [82]:

dfUS['Last_Update']         = pd.to_datetime(dfUS['Last_Update'])
dfUS['File_Date']           = pd.to_datetime(dfUS['File_Date'])
dfUS["FIPS"]                = pd.to_numeric(dfUS["FIPS"])
dfUS["Confirmed"]           = pd.to_numeric(dfUS["Confirmed"])
dfUS["Deaths"]              = pd.to_numeric(dfUS["Deaths"])
dfUS["Recovered"]           = pd.to_numeric(dfUS["Recovered"])
dfUS["Active"]              = pd.to_numeric(dfUS["Active"])
dfUS["Latitude"]            = pd.to_numeric(dfUS["Latitude"])
dfUS["Longitude"]           = pd.to_numeric(dfUS["Longitude"])
dfUS["Incident_Rate"]       = pd.to_numeric(dfUS["Incident_Rate"])

dfUS["Total_Test_Results"]   = pd.to_numeric(dfUS["Total_Test_Results"])
dfUS["People_Hospitalized"]  = pd.to_numeric(dfUS["People_Hospitalized"])
dfUS["UID"]                  = pd.to_numeric(dfUS["UID"])
dfUS["Testing_Rate"]         = pd.to_numeric(dfUS["Testing_Rate"])
dfUS["Hospitalization_Rate"] = pd.to_numeric(dfUS["Hospitalization_Rate"])
dfUS["People_Tested"]        = pd.to_numeric(dfUS["People_Tested"])
dfUS["Mortality_Rate"]       = pd.to_numeric(dfUS["Mortality_Rate"])
dfUS["Cases_28_Days"]        = pd.to_numeric(dfUS["Cases_28_Days"])
dfUS["Deaths_28_Days"]       = pd.to_numeric(dfUS["Deaths_28_Days"])

SetStringType(dfUS, "Province_State", "Country_Region", "ISO3")


## Check for nulls

#### Reports

In [83]:
df.isnull().sum()

FIPS                   392274
Admin2                   8882
Province_State              0
Country_Region              0
Last_Update                 0
Latitude                50278
Longitude               50278
Confirmed                  28
Deaths                    434
Recovered              750546
Active                 759891
Combined_Key             8882
Incident_Rate          268767
Case_Fatality_Ratio    225348
File_Date                   0
dtype: int64

### So, for us the most important information is the "Confirmed", "Deaths", "Recovered", "Active" fields. Since they have null values, we get rid of the corresponding lines 

In [46]:
df = df.dropna(subset=['Confirmed'])
df = df.dropna(subset=['Deaths'])
df = df.dropna(subset=['Recovered'])
df = df.dropna(subset=['Active'])

#### Reports_US

In [84]:
dfUS.isnull().sum()

FIPS                       19
Province_State              0
Country_Region              0
Last_Update                19
Latitude                 1106
Longitude                1106
Confirmed                   0
Deaths                      0
Recovered               16448
Active                  12487
Incident_Rate            1106
Case_Fatality_Ratio     12256
Total_Test_Results      12940
People_Hospitalized     26441
UID                         0
ISO3                        0
Testing_Rate             1124
Hospitalization_Rate    26441
People_Tested           19754
Mortality_Rate          19543
Cases_28_Days           31454
Deaths_28_Days          31454
File_Date                   0
dtype: int64

### Similar situation here

In [85]:
dfUS = dfUS.dropna(subset=['Confirmed'])
dfUS = dfUS.dropna(subset=['Deaths'])
dfUS = dfUS.dropna(subset=['Recovered'])
dfUS = dfUS.dropna(subset=['Active'])

## Сhecking for abnormal values

#### Reports

#### FIPS - Alright

In [86]:
print('Min: ', df['FIPS'].min())
print('Max: ', df['FIPS'].max())

Min:  60.0
Max:  99999.0


#### Last Update - Alright

In [87]:
print('Min: ', df['Last_Update'].min())
print('Max: ', df['Last_Update'].max())

Min:  2020-01-22 17:00:00
Max:  2021-10-08 04:21:25


#### Confirmed - No way

In [88]:
print('Min: ', df['Confirmed'].min())
print('Max: ', df['Confirmed'].max())

Min:  -302844.0
Max:  7357306.0


In [89]:
df = df.drop(df[df.Confirmed < 0].index)

#### Deaths - No way

In [90]:
print('Min: ', df['Deaths'].min())
print('Max: ', df['Deaths'].max())

Min:  -178.0
Max:  150540.0


In [56]:
df = df.drop(df[df.Deaths < 0].index)

#### Recovered - No way

In [91]:
print('Min: ', df['Recovered'].min())
print('Max: ', df['Recovered'].max())

Min:  -854405.0
Max:  6399531.0


In [92]:
df = df.drop(df[df.Recovered < 0].index)

#### Active - No way

In [59]:
print('Min: ', df['Active'].min())
print('Max: ', df['Active'].max())

Min:  -6298082.0
Max:  10877758.0


In [93]:
df = df.drop(df[df.Active < 0].index)

#### Reports-US

#### FIPS - Alright

In [94]:
print('Min: ', dfUS['FIPS'].min())
print('Max: ', dfUS['FIPS'].max())

Min:  1.0
Max:  99999.0


#### Last Update - Alright

In [95]:
print('Min: ', dfUS['Last_Update'].min())
print('Max: ', dfUS['Last_Update'].max())

Min:  2020-04-12 23:18:15
Max:  2021-03-07 05:31:30


#### Confirmed - Alright

In [96]:
print('Min: ', dfUS['Confirmed'].min())
print('Max: ', dfUS['Confirmed'].max())

Min:  0
Max:  2692691


#### Deaths - Alright

In [97]:
print('Min: ', dfUS['Deaths'].min())
print('Max: ', dfUS['Deaths'].max())

Min:  0
Max:  46729


#### Recovered - Alright

In [98]:
print('Min: ', dfUS['Deaths'].min())
print('Max: ', dfUS['Deaths'].max())

Min:  0
Max:  46729


#### Active- Alright

In [99]:
print('Min: ', dfUS['Active'].min())
print('Max: ', dfUS['Active'].max())

Min:  -120720.0
Max:  1408321.0


## Unconnection

In [100]:
cur.close()
con.close()